In [1]:
# =====================================================
# XGBoost - Asymmetric Loss
# Walk-Forward Day-Ahead Forecast
# Electricity Market - Declared Power
# =====================================================

import pandas as pd
import numpy as np
import xgboost as xgb

# =========================
# تنظیمات
# =========================
INPUT_FILE = "merged_output2.csv"
OUTPUT_FILE = "declared_xgb_asymmetric.xlsx"

TARGET = "POWER"
DATE_COL = "DATE_MILADI"
HOUR_COL = "HOUR"
EBRAZ_COL = "ebraz"

ALPHA = 1.0     # وزن کم‌ابرازی
BETA = 5.0      # وزن بیش‌ابرازی (جریمه)
ANCHOR_WEIGHT = 0.5
MIN_RATIO = 0.75

# =========================
# خواندن داده
# =========================
df = pd.read_csv(INPUT_FILE)
df[DATE_COL] = pd.to_datetime(df[DATE_COL])

df = df.sort_values([DATE_COL, HOUR_COL]).reset_index(drop=True)

# =========================
# ویژگی‌های زمانی
# =========================
df["hour"] = df[HOUR_COL]
df["dayofweek"] = df[DATE_COL].dt.dayofweek
df["month"] = df[DATE_COL].dt.month

# =========================
# Lag Features
# =========================
df["lag_24"] = df[TARGET].shift(24)
df["lag_48"] = df[TARGET].shift(48)
df["lag_72"] = df[TARGET].shift(72)

df = df.dropna().reset_index(drop=True)

FEATURES = [
    "hour",
    "dayofweek",
    "month",
    "DAMA",
    "ROTOOBAT",
    "lag_24",
    "lag_48",
    "lag_72"
]

# =========================
# Loss نامتقارن
# =========================
def asymmetric_loss(alpha, beta):
    def loss(y_true, y_pred):
        error = y_pred - y_true
        grad = np.where(
            error > 0,
            2 * beta * error,     # بیش‌ابرازی
            2 * alpha * error     # کم‌ابرازی
        )
        hess = np.where(
            error > 0,
            2 * beta,
            2 * alpha
        )
        return grad, hess
    return loss

# =========================
# Walk-Forward Day-Ahead
# =========================
df["DECLARED"] = np.nan
unique_days = df[DATE_COL].dt.date.unique()

for i in range(3, len(unique_days) - 1):

    train_days = unique_days[:i]
    predict_day = unique_days[i]

    train_idx = df[DATE_COL].dt.date.isin(train_days)
    test_idx = df[DATE_COL].dt.date == predict_day

    X_train = df.loc[train_idx, FEATURES]
    y_train = df.loc[train_idx, TARGET]

    X_test = df.loc[test_idx, FEATURES]

    model = xgb.XGBRegressor(
        n_estimators=400,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective=asymmetric_loss(ALPHA, BETA),
        random_state=42
    )

    model.fit(X_train, y_train)

    preds = model.predict(X_test)

    # ---------- Anchor به lag_24 ----------
    lag24 = df.loc[test_idx, "lag_24"].values
    declared = (
        (1 - ANCHOR_WEIGHT) * preds +
        ANCHOR_WEIGHT * lag24
    )

    # ---------- کف محافظ ----------
    declared = np.maximum(declared, lag24 * MIN_RATIO)

    df.loc[test_idx, "DECLARED"] = declared

# =========================
# منطق بازار برق
# =========================
df.loc[df[EBRAZ_COL] == 0, "DECLARED"] = 0
df["DECLARED"] = df["DECLARED"].clip(lower=0)

# =========================
# ارزیابی
# =========================
err = df["DECLARED"] - df[TARGET]

mae_pos = np.mean(np.abs(err[err > 0]))   # بیش‌ابرازی
mae_neg = np.mean(np.abs(err[err < 0]))   # کم‌ابرازی

print("===================================")
print("XGBoost Asymmetric Loss")
print("===================================")
print("MAE Positive (Over):", round(mae_pos, 2))
print("MAE Negative (Under):", round(mae_neg, 2))
print("Market Score:", round(5 * mae_pos + mae_neg, 2))

# =========================
# ذخیره
# =========================
df.to_excel(OUTPUT_FILE, index=False)
print("Saved:", OUTPUT_FILE)


XGBoost Asymmetric Loss
MAE Positive (Over): 13.27
MAE Negative (Under): 12.34
Market Score: 78.71
Saved: declared_xgb_asymmetric.xlsx
